In [ ]:
# Force reload all modules to clear caches
import sys
import importlib

# Remove all multi_agent_system modules from cache
keys_to_delete = [key for key in list(sys.modules.keys()) if 'multi_agent_system' in key or 'langchain' in key or 'langgraph' in key or 'tavily' in key]
for key in keys_to_delete:
    try:
        del sys.modules[key]
    except:
        pass

print(f"Removed {len(keys_to_delete)} cached modules")

# Also clear importlib caches
importlib.invalidate_caches()
print("Invalidated importlib caches")

In [ ]:
import sys

# Final cache clear before imports
keys_to_delete = [key for key in list(sys.modules.keys()) if 'multi_agent_system' in key]
for key in keys_to_delete:
    try:
        del sys.modules[key]
    except:
        pass

# Ensure we're in the right path
sys.path.insert(0, '/Users/lixuepeng03/Jack-Investment-Assistant')

from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore
from langgraph_supervisor import create_supervisor

from multi_agent_system.config import LLM_CONFIG, SUPERVISOR_PROMPT
from multi_agent_system.state import State
from multi_agent_system.searcher_agent import create_searcher_agent
from multi_agent_system.summary_agent import create_summary_agent
from multi_agent_system.utils import show_graph

"""
Create and compile the supervisor agent.
    
Returns:
    langgraph.graph: Compiled supervisor agent graph.
"""
# Initialize the LLM
llm = ChatOpenAI(**LLM_CONFIG)
    
# Initialize memory components
checkpointer = MemorySaver()
in_memory_store = InMemoryStore()

# Create sub-agents
searcher_agent = create_searcher_agent()
summary_agent = create_summary_agent()
# coder_agent = create_coder_agent()
    

# Import the supervisor creator to avoid name collision
from langgraph_supervisor import create_supervisor as create_supervisor_workflow

supervisor_prebuilt_workflow = create_supervisor_workflow(
        agents=[searcher_agent, summary_agent],
        output_mode="last_message",
        model=llm,
        prompt=SUPERVISOR_PROMPT,
        state_schema=State
    )
        
supervisor_prebuilt = supervisor_prebuilt_workflow.compile(
        name="supervisor",
        checkpointer=checkpointer,
        store=in_memory_store
    )
    
show_graph(supervisor_prebuilt)

In [ ]:
import sys

# Final cache clear before imports
keys_to_delete = [key for key in list(sys.modules.keys()) if 'multi_agent_system' in key]
for key in keys_to_delete:
    try:
        del sys.modules[key]
    except:
        pass

# Ensure we're in the right path
sys.path.insert(0, '/Users/lixuepeng03/Jack-Investment-Assistant')


from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore
from langgraph_supervisor import create_supervisor

from multi_agent_system.config import LLM_CONFIG, SUPERVISOR_PROMPT
from multi_agent_system.state import State
from multi_agent_system.searcher_agent import create_searcher_agent
from multi_agent_system.summary_agent import create_summary_agent
from multi_agent_system.utils import show_graph


def create_supervisor():
    """
    Create and compile the supervisor agent.
    
    Returns:
        langgraph.graph: Compiled supervisor agent graph.
    """
    # Initialize the LLM
    llm = ChatOpenAI(**LLM_CONFIG)
    
    # Initialize memory components
    checkpointer = MemorySaver()
    in_memory_store = InMemoryStore()

    # Create sub-agents
    searcher_agent = create_searcher_agent()
    summary_agent = create_summary_agent()
    # coder_agent = create_coder_agent()
    

    # Import the supervisor creator to avoid name collision
    from langgraph_supervisor import create_supervisor as create_supervisor_workflow

    supervisor_prebuilt_workflow = create_supervisor_workflow(
            agents=[searcher_agent, summary_agent],
            output_mode="last_message",
            model=llm,
            prompt=SUPERVISOR_PROMPT,
            state_schema=State
        )
        
    supervisor_prebuilt = supervisor_prebuilt_workflow.compile(
            name="supervisor",
            checkpointer=checkpointer,
            store=in_memory_store
        )
    
    show_graph(supervisor_prebuilt)
    
    return supervisor_prebuilt


if __name__ == "__main__":
    """Test the supervisor independently."""
    import uuid
    from langchain_core.messages import HumanMessage
    
    # Create the supervisor
    supervisor = create_supervisor()
    
    thread_id = uuid.uuid4()
    question = ""
    config = {"configurable": {"thread_id": thread_id}}
    
    print(f"Thread ID: {thread_id}")
    print(f"Question: {question}\n")
    
    result = supervisor.invoke(
        {"messages": [HumanMessage(content=question)]},
        config=config
    )
    
    print("Response:")
    for message in result["messages"]:
        message.pretty_print()


In [ ]:
"""Summary agent for web search and information retrieval."""
import sys

# Final cache clear before imports
keys_to_delete = [key for key in list(sys.modules.keys()) if 'multi_agent_system' in key]
for key in keys_to_delete:
    try:
        del sys.modules[key]
    except:
        pass

# Ensure we're in the right path
sys.path.insert(0, '/Users/lixuepeng03/Investment-Assistant')


from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore
from multi_agent_system.state import State
from multi_agent_system.utils import show_graph
from multi_agent_system.memory_tool import load_memory
from multi_agent_system.supervisor import create_supervisor
from multi_agent_system.memory_agent import create_memory
from langgraph.checkpoint.memory import MemorySaver # For short-term memory (thread-level state persistence)
from langgraph.store.memory import InMemoryStore # For long-term memory (storing user preferences)
from multi_agent_system.config import LLM_CONFIG
from langgraph.store.base import BaseStore # Base class for defining custom stores for LangGraph

llm = ChatOpenAI(**LLM_CONFIG)

in_memory_store = InMemoryStore()

checkpointer = MemorySaver()

invest_assistan_agent = StateGraph(State)

invest_assistan_agent.add_node("load_memory", load_memory) 
invest_assistan_agent.add_node("supervisor", create_supervisor)  # Call the function to get the compiled graph
invest_assistan_agent.add_node("create_memory", create_memory)

invest_assistan_agent.add_edge(START, "load_memory")
invest_assistan_agent.add_edge("load_memory", "supervisor")
invest_assistan_agent.add_edge("supervisor", "create_memory")
invest_assistan_agent.add_edge("create_memory", END)

# Compile the entire, sophisticated graph.
invest_assistant_workflow = invest_assistan_agent.compile(name="multi_agent_verify", checkpointer=checkpointer, store=in_memory_store)

show_graph(invest_assistant_workflow)


In [ ]:
"""Summary agent for web search and information retrieval."""
import sys

# Final cache clear before imports
keys_to_delete = [key for key in list(sys.modules.keys()) if 'multi_agent_system' in key]
for key in keys_to_delete:
    try:
        del sys.modules[key]
    except:
        pass

# Ensure we're in the right path
sys.path.insert(0, '/Users/lixuepeng03/Investment-Assistant')


from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore
from multi_agent_system.state import State
from multi_agent_system.utils import show_graph
from multi_agent_system.memory_tool import load_memory
from multi_agent_system.supervisor import create_supervisor
from multi_agent_system.memory_agent import create_memory
from langgraph.checkpoint.memory import MemorySaver # For short-term memory (thread-level state persistence)
from langgraph.store.memory import InMemoryStore # For long-term memory (storing user preferences)
from multi_agent_system.config import LLM_CONFIG
from langgraph.store.base import BaseStore # Base class for defining custom stores for LangGraph



def create_invest_assistant():
    """Creates a multi-agent investment assistant with memory management."""
     # Initialize the LLM
    llm = ChatOpenAI(**LLM_CONFIG)

    in_memory_store = InMemoryStore()

    checkpointer = MemorySaver()

    invest_assistan_agent = StateGraph(State)

    invest_assistan_agent.add_node("load_memory", load_memory) 
    invest_assistan_agent.add_node("supervisor", create_supervisor)
    invest_assistan_agent.add_node("create_memory", create_memory)

    invest_assistan_agent.add_edge(START, "load_memory")
    invest_assistan_agent.add_edge("load_memory", "supervisor")
    invest_assistan_agent.add_edge("supervisor", "create_memory")
    invest_assistan_agent.add_edge("create_memory", END)

    # Compile the entire, sophisticated graph.
    invest_assistant_workflow = invest_assistan_agent.compile(name="multi_agent_verify", checkpointer=checkpointer, store=in_memory_store)
    
    return invest_assistant_workflow

if __name__ == "__main__":
    """Test the invest assistant independently."""
    import uuid
    from langchain_core.messages import HumanMessage
    
    # Create the invest assistant
    invest_assistant = create_invest_assistant()

    show_graph(invest_assistant)

    thread_id = uuid.uuid4()
    config = {"configurable": {"thread_id": thread_id, "user_id" : "5"}}

    question = ""

    result = invest_assistant.invoke({"messages": [HumanMessage(content=question)]}, config=config)

    # Print results
    print("Response:")
    result["messages"][-1].pretty_print()
    # for message in result["messages"]:
    #     message.pretty_print() 


    print("---------分割线----------\n")

    

    question2 = ""
    result2 = invest_assistant.invoke({"messages": [HumanMessage(content=question2)]}, config=config)
    

    # Print results
    print("Response:")
    result2["messages"][-1].pretty_print()
    # for message in result2["messages"]:
    #     message.pretty_print() 
    
    
